### PSE GYM- ASU 

In [1]:
# from product_management import Product_Management, doslfp_optimizier
# main.py
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.spatial import ConvexHull
import random
from pyomo.environ import ConcreteModel, Set, Param, Block
import pyomo.environ as pyo
import pyomo.opt as opt
solver = pyo.SolverFactory('gurobi')
import math
from math import ceil
import csv
import json
import time
from math import sqrt
import copy
import ast

### ASU Simulation

In [ ]:
from ASU_opt_model import optimize_ASU
from ASU_env import ASUEnv
days_to_simulate = 3

start_day = 1
lookahead = 4
env_id = 'ASU1'
# env = ASUEnv('ASU1', lookahead, days_to_simulate)
env = ASUEnv('ASU1')
asu_optimizer = optimize_ASU('ASU1', lookahead)

env.reset()
current_day = 1
optimization_day1_cost = {}
opt_lambda = {}
opt_reward = {}

terminated = False
while not terminated:

    # state = env.render()
    state = env._get_state(mode='dict')
    asu_optimizer.update_state(state)
    
    # Solve the optimization model
    objective = asu_optimizer.solve()
    day1_cost = asu_optimizer.day1_cost()
    optimization_day1_cost[current_day] = day1_cost  
    
    opt_day_lambda = asu_optimizer.extract_optimal_lambda()
    # fetch optimal hourly lambda values for the current day
    opt_lambda[current_day] = opt_day_lambda
    # based on opt_day_lambda, calculate the reward: opt_reward[current_day]
    
    day_reward = 0.0
    for hour in range(1, 25):
        # Retrieve the lambda values for this hour (assume keys in subdict are sorted correctly, e.g., 1, 2, ...).
        hourly_lambda = opt_day_lambda[hour]
        # Convert the lambda dict to a NumPy array. # For instance, if your extreme points indices are [1,2,...,n], sort the keys and build the array.
        lambda_array = np.array([hourly_lambda[k] for k in sorted(hourly_lambda.keys())], dtype=np.float32)
        # opt_action = {'lambda': lambda_array}
        opt_action = lambda_array
        
        # Take a step in the environment using this optimal action.
        # state, reward, done, info = env.step(opt_action)
        _, reward, terminated, _ , _ = env.step(opt_action)
        terminated = terminated.item()
        day_reward += reward.item()

    opt_reward[current_day] = day_reward

    # Approaching end of current day
    if terminated == True:
        print("Simulation complete for total of {} days".format(current_day))
    current_day += 1

Simulation complete for total of 7 days


In [3]:
print(optimization_day1_cost)
print(opt_reward)

{1: 1044.5566829738332, 2: 861.3592639213889, 3: 805.6513215031631, 4: 1007.3574626449497, 5: 887.7661076585582, 6: 927.1291526642696, 7: 1063.684094797056}
{1: tensor(-1044.5566), 2: tensor(-861.3593), 3: tensor(-805.6513), 4: tensor(-1007.3574), 5: tensor(-887.7661), 6: tensor(-927.2236), 7: tensor(-1063.6841)}


In [4]:
state

{'electricity_prices': array([0.11611212, 0.11472997, 0.11304277, 0.11281865, 0.1137194 ,
        0.11568954, 0.11940014, 0.11715174, 0.11393   , 0.11809806,
        0.11997571, 0.12877017, 0.1448933 , 0.15464   , 0.1508881 ,
        0.15561247, 0.15824   , 0.153014  , 0.14484   , 0.14967705,
        0.13849321, 0.12562515, 0.12105219, 0.11920443, 0.1157513 ,
        0.11465624, 0.11432514, 0.1132992 , 0.112     , 0.11353   ,
        0.11827928, 0.11662449, 0.11612549, 0.11834417, 0.12291305,
        0.13000876, 0.13380855, 0.14596574, 0.14826503, 0.16292158,
        0.17346975, 0.15394147, 0.14713089, 0.18594   , 0.184194  ,
        0.13900852, 0.12788   , 0.1239281 , 0.11509209, 0.11486375,
        0.11290072, 0.11006188, 0.11440136, 0.11074453, 0.11505302,
        0.11131887, 0.10905628, 0.11388979, 0.11524954, 0.1168518 ,
        0.12181839, 0.1287396 , 0.13143963, 0.13365646, 0.14148454,
        0.13885   , 0.14197975, 0.15070295, 0.14076486, 0.13115154,
        0.12039614, 0.1165

In [5]:
def encode_observation(state):
        """
        Encode the observation into a flattened format.
        This method converts the state dictionary into a single array.
        """
        # Flatten the state dictionary into a single array
        flatt_state = np.concatenate([
            state['electricity_prices'],
            state['demand'].flatten(),
            state['IV']
        ])
        return flatt_state

In [ ]:
flatt_state = encode_observation(state)
print(flatt_state.shape)

(483,)


In [7]:
24*5 + 24*3*5 + 3


483